# QuickStart

_ActionWeaver is an AI application framework that puts function-calling as a first-class feature_.

---

This demonstrates how to seamlessly integrate any Python function into your LLM application using ActionWeaver and either the Azure or OpenAI API.
 

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Use ActionWeaver and OpenAI API**

In [6]:
from actionweaver.utils.tokens import TokenUsageTracker
from actionweaver.llms import patch
from openai import OpenAI

openai_client = patch(OpenAI())

In [7]:
from actionweaver import action

@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    print ("Getting current time...")
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"


@action(name="GetWeather", stop=False)
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    print ("Getting current weather")
    
    import json
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [8]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what's the weather in San Francisco and Beijing ?"}
  ]


response = openai_client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
    actions = [get_current_weather],
    stream=False, 
    token_usage_tracker = TokenUsageTracker(500),
)

response

Getting current weather
Getting current weather


ChatCompletion(id='chatcmpl-8olQcSJWBqrdiBevxOTSVt0n0qS19', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The current weather in San Francisco is 72°F. Unfortunately, I couldn't retrieve the current weather for Beijing.", role='assistant', function_call=None, tool_calls=None))], created=1707109406, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=24, prompt_tokens=153, total_tokens=177))

**Use ActionWeaver and Azure OpenAI Service**

In [9]:
import os
from openai import AzureOpenAI

azure_client = patch(AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2023-10-01-preview"
))

In [10]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what's the weather in San Francisco and Beijing ?"}
  ]

response = azure_client.chat.completions.create(
  model="gpt-35-turbo-0613-16k",
  messages=messages,
  stream=False,
  actions = [get_current_weather],
  token_usage_tracker = TokenUsageTracker(500),
)

response

Getting current weather
Getting current weather


ChatCompletion(id='chatcmpl-8olQhd7zmsGYXe0U5p7Pv7lbWF1re', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The current temperature in San Francisco is 72°F. However, I couldn't retrieve the current temperature in Beijing at the moment.", role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1707109411, model='gpt-35-turbo-16k', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=27, prompt_tokens=155, total_tokens=182), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'f

**Easily integrate tools from libraries such as [Langchain](https://github.com/langchain-ai/langchain/tree/master/libs/community)**

In [11]:
from langchain_community.tools.google_search.tool import GoogleSearchRun
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
search_tool = GoogleSearchRun(api_wrapper=GoogleSearchAPIWrapper())

In [12]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "search what is ActionWeaver on Github"}
  ]

from actionweaver.actions.factories.langchain import action_from_tool

response = azure_client.chat.completions.create(
  model="gpt-35-turbo-0613-16k",
  messages=messages,
  stream=False,
  actions = [action_from_tool(search_tool, description="Invoke this tool to search any information")],
  token_usage_tracker = TokenUsageTracker(500),
)

response

ChatCompletion(id='chatcmpl-8olQkSd1ng4l4x1IdEkrkgmiYHGnN', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='ActionWeaver is an AI application framework that is designed based on the concept of LLM (Language Learning Model) function calling. It is built with the aim of scaling up and orchestrating OpenAI functions. With ActionWeaver, you can easily turn any function into an OpenAI language model function using a simple decorator.\n\nThe ActionWeaver project is available on GitHub at: [https://github.com/TengHu/ActionWeaver](https://github.com/TengHu/ActionWeaver)', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1707109414, model='gpt-35-turbo-16k', object='chat.completion', system_fingerp

**Force execute an action**

In [13]:
get_current_time.invoke(openai_client, messages=[{"role": "user", "content": "what time"}], model="gpt-3.5-turbo", stream=False, force=False)


Getting current time...


ChatCompletion(id='chatcmpl-8olQpnI7bvn6K9HVOWHJzYwLa2Ze4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current time is 00:03:39.', role='assistant', function_call=None, tool_calls=None))], created=1707109419, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=98, total_tokens=110))

In [14]:
get_current_time.invoke(azure_client, messages=[{"role": "user", "content": "what time"}], model="gpt-35-turbo-0613-16k", stream=False, force=False)


Getting current time...


ChatCompletion(id='chatcmpl-8olQsrtqkktjMNjXRj1KBItN9uwaY', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current time is 00:03:41.', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1707109422, model='gpt-35-turbo-16k', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=98, total_tokens=110), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])

**Stop the Action in the loop**

Every action comes with a stop argument, which is set to False by default, if True this means that the LLM will immediately return the function's output if chosen, but this also restricts the LLM from making multiple function calls. For instance, if asked about the weather in NYC and San Francisco, the model would invoke two separate functions sequentially for each city. However, with `stop=True`, this process is interrupted once the first function returns weather information for either NYC or San Francisco, depending on which city it queries first.

In [15]:
get_current_weather.stop = True

In [19]:
get_current_weather.invoke(openai_client, messages=[{"role": "user", "content": "what weather is San Francisco"}], model="gpt-3.5-turbo", stream=False, force=True)


Getting current weather


['{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}']